In [1]:
# import the dependencies
import pandas as pd
import gmaps
import requests
# import the api key
from config import g_key

In [2]:
# store the csv saved and created in part 1 into a dataframe
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,2022-03-19 01:52:21,-46.1927,168.8643,72.48,41,100,5.68
1,1,Bredasdorp,ZA,2022-03-19 01:52:21,-34.5322,20.0403,66.97,96,62,7.63
2,2,Berbera,SO,2022-03-19 01:52:22,10.4396,45.0143,74.91,74,1,3.96
3,3,Ushuaia,AR,2022-03-19 01:52:22,-54.8000,-68.3000,38.86,75,40,9.22
4,4,Ternate,ID,2022-03-19 01:52:22,0.8000,127.4000,85.03,69,100,1.66
5,5,Avarua,CK,2022-03-19 01:52:23,-21.2078,-159.7750,84.25,74,70,5.75
6,6,Victoria,HK,2022-03-19 01:47:50,22.2855,114.1577,79.97,87,50,15.99
7,7,Little Current,CA,2022-03-19 01:52:23,45.9793,-81.9248,41.05,92,100,12.53
8,8,Yulara,AU,2022-03-19 01:52:24,-25.2406,130.9889,91.24,24,40,8.05
9,9,Tuatapere,NZ,2022-03-19 01:52:24,-46.1333,167.6833,68.47,47,100,4.38


In [3]:
# get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# configure gmaps to use google api key
gmaps.configure(api_key=g_key)

In [5]:
# get the maximum temperature (clean the negative temperature)

max_temp = city_data_df['Max Temp']
temps = []
for temp in max_temp:
    temps.append(max(temp,0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
max_temp = city_data_df['Max Temp']
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# ask the user to add a min and max temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# filter the dataset to find the cities that fit the criteria
prefered_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]

prefered_cities_df.count()

City_ID       140
City          140
Country       140
Date          140
Lat           140
Lng           140
Max Temp      140
Humidity      140
Cloudiness    140
Wind Speed    140
dtype: int64

In [14]:
# create a dataframe called hotel_df to store hotel names along with city, country, temp, and coordinates
hotel_df = prefered_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Ternate,ID,85.03,0.8000,127.4000,
5,Avarua,CK,84.25,-21.2078,-159.7750,
6,Victoria,HK,79.97,22.2855,114.1577,
12,Carnarvon,AU,82.47,-24.8667,113.6333,
15,Hithadhoo,MV,83.12,-0.6000,73.0833,


In [17]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# iterate through the Data frame
for index, row in hotel_df.iterrows():
    # get the lat and long
    lat = row['Lat']
    lng = row['Lng']
    
    # add the lat and long to location key for the params dictionary
    params['location'] = f"{lat}, {lng}"
    
    # use the search term: lodging and our lat and long
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make the request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    # grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found...skipping.")
hotel_df.head(10)

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Ternate,ID,85.03,0.8000,127.4000,Sahid Bela Ternate Hotel
5,Avarua,CK,84.25,-21.2078,-159.7750,Paradise Inn
6,Victoria,HK,79.97,22.2855,114.1577,Mini Hotel Central
12,Carnarvon,AU,82.47,-24.8667,113.6333,Hospitality Carnarvon
15,Hithadhoo,MV,83.12,-0.6000,73.0833,Scoop Guest House
18,Vaini,TO,81.50,-21.2000,-175.2000,Keleti Beach Resort
19,Cidreira,BR,77.05,-30.1811,-50.2056,Hotel Castelo
21,Andros Town,BS,76.96,24.7000,-77.7667,Hanks place
25,Buala,SB,84.36,-8.1450,159.5921,Maringe Lagoon Lodge
29,Georgetown,MY,87.73,5.4112,100.3354,Cititel Penang


In [19]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))